In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras
import numpy as np

In [3]:
mnist = keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train = np.reshape(x_train,[-1,784])

In [6]:
x_test = np.reshape(x_test,[-1,784])

In [43]:
#设置类别的数量
#     num_classes = 10
#     #需要转换的整数
#     arr = [1,3,4,5,9]
#     #将整数转为一个10位的one hot编码
#     print(np.eye(10)[arr])

def convert_to_one_hot(y,num_class):
    return np.eye(num_class)[y]

In [8]:
np.eye(3)[[1,2]]

array([[0., 1., 0.],
       [0., 0., 1.]])

In [44]:
y_train = convert_to_one_hot(y_train,10)

In [47]:
y_test = convert_to_one_hot(y_test,10)

In [48]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_train.astype('float32')

In [13]:
X = tf.placeholder(tf.float32,[None,784])
Y = tf.placeholder(tf.float32,[None,10])

In [14]:
# 初始化权重参数
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [15]:
w_h = weight_variable([784,625])
w_h2 = weight_variable([625,625])
w_o = weight_variable([625,10])

b_h = bias_variable([625])
b_h2 = bias_variable([625])
b_o = bias_variable([10])

In [16]:
def model(X,w_h,w_h2,w_o,b_h,b_h2,b_o,p_keep_input,p_keep_hidden):
    X = tf.nn.dropout(X,p_keep_input)
    h = tf.nn.relu(tf.matmul(X,w_h)+b_h)
    h = tf.nn.dropout(h,p_keep_hidden)
    
    h2 = tf.nn.relu(tf.matmul(h,w_h2)+b_h2)
    h2 = tf.nn.dropout(h2,p_keep_hidden)
    
    return tf.matmul(h2,w_o)+b_o

In [17]:
p_keep_input = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)
py_x = model(X,w_h,w_h2,w_o,b_h,b_h2,b_o,p_keep_input,p_keep_hidden)

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=py_x,labels=Y))
train_op = tf.train.RMSPropOptimizer(0.01,0.9).minimize(cost)
predict_op = tf.argmax(py_x,1)

In [20]:
import os
ckpt_dir = "./ckpt_dir" 
if not os.path.exists( ckpt_dir): 
    os.makedirs( ckpt_dir)

In [21]:
# 计数器 变量， 设置 它的 trainable= False， 不需 要被 训练 
global_step = tf.Variable( 0, name='global_step', trainable= False)

In [22]:
# 在 声明 完 所有 变量 后， 调用 tf.train.Saver 
saver = tf.train.Saver() # 位于 tf. train. Saver 之后 的 变量 将不 会被 存储 
non_storable_variable = tf.Variable(777)

In [ ]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    start = global_step.eval() # 得到 global_ step 的 初始 值 
    print(" Start from:", start) 
    for i in range( start, 100): # 以 128 作为 batch_ size 
        for start, end in zip( range( 0, 60000, 128), range( 128, 60000+ 1, 128)): 
            sess.run( train_op, feed_dict={ X: x_train[start:end], Y: y_train[start:end], p_keep_input: 0.8, p_keep_hidden: 0.5}) 
            global_step.assign(i).eval() # 更新 计数器 
            saver.save( sess, ckpt_dir + "/model.ckpt", global_step= global_step) # 存储 模型
    

 Start from: 0


In [25]:
y_train[start:end].shape

(128,)